In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import statistics
import scipy.stats as sts
n = 16
a = -1
gamma = 0.9
sigma = 4

In [2]:
sample = np.random.normal(a, sigma, n)#создание выборки
mean = statistics.mean(sample)

for i in range(0,20):
    t = 1.5 + i*0.01
    F_s =  sts.norm().cdf(t)
    print (f't = {t} F_s={F_s}')

t = 1.5 F_s=0.9331927987311419
t = 1.51 F_s=0.9344782879110836
t = 1.52 F_s=0.9357445121810641
t = 1.53 F_s=0.9369916355360216
t = 1.54 F_s=0.9382198232881881
t = 1.55 F_s=0.939429241997941
t = 1.56 F_s=0.940620059405207
t = 1.57 F_s=0.941792444361447
t = 1.58 F_s=0.9429465667622459
t = 1.59 F_s=0.9440825974805306
t = 1.6 F_s=0.945200708300442
t = 1.61 F_s=0.9463010718518803
t = 1.62 F_s=0.9473838615457479
t = 1.63 F_s=0.9484492515099107
t = 1.6400000000000001 F_s=0.9494974165258963
t = 1.65 F_s=0.9505285319663519
t = 1.66 F_s=0.9515427737332772
t = 1.67 F_s=0.9525403181970526
t = 1.68 F_s=0.9535213421362799
t = 1.69 F_s=0.9544860226784502


In [3]:
t2 = (1 + gamma)/2
t =  sts.norm().ppf(t2)
print(t)

1.6448536269514722


In [4]:
delta = (t*sigma) / np.sqrt(n)
I_left = mean - delta
I_right = mean + delta
print(I_left,I_right)

-2.7036061121426407 0.5861011417603039


In [6]:
interval = sts.norm.interval(gamma, a , sigma/math.sqrt(n))
print(interval)

(-2.644853626951473, 0.6448536269514722)


In [5]:
count = 0.0
M = 1000
for j in range(0,M):
    sample = np.random.normal(a, sigma, n)#создание выборки
    mean = statistics.mean(sample)

    interval = sts.norm.interval(gamma, mean, sigma/math.sqrt(n))
    
    if interval[0]<a and interval[1]>a:
        count +=1

GAMMA = count/M
print(GAMMA)

0.899
